In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

url = "https://www.nba.com/stats/players/traditional?Season=2020-21"
driver.get(url)

driver.implicitly_wait(10)

try:
    dropdown = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//select[@name='PerPage']"))
    )
    # Select the "All" option (update value if different)
    driver.find_element(By.XPATH, "//select[@name='PerPage']/option[text()='All']").click()
    print("Selected 'All' players per page.")
except:
    try:
        driver.find_element(By.XPATH, "//select[@name='PerPage']/option[@value='540']").click()
        print("Selected max value (e.g., 540) players per page.")
    except:
        print("Couldn’t find 'All' or max value in dropdown. Scraping default page size.")

WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.TAG_NAME, "tbody"))
)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table', class_='Crom_table__p1iZz')

if not table:
    print("Table not found.")
    driver.quit()
    exit()

thead = table.find('thead')
if thead:
    header_elements = thead.find_all('th')
    header_names = [th.text.strip() for th in header_elements]
else:
    header_names = ['Rank', 'Player', 'Team', 'Age', 'GP', 'W', 'L', 'Min', 'PTS', 'FGM', 
                    'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 
                    'REB', 'AST', 'TOV', 'STL', 'BLK', 'PF', 'FP', 'DD2', 'TD3', '+/-']

tbody = table.find('tbody')
if not tbody:
    print("No tbody found.")
    driver.quit()
    exit()

rows = []
for tr in tbody.find_all('tr'):
    row = [td.text.strip() for td in tr.find_all('td')]
    if row:
        rows.append(row)

if not rows:
    print("No rows found.")
    driver.quit()
    exit()

if len(header_names) != len(rows[0]):
    print(f"Header length ({len(header_names)}) doesn’t match row length ({len(rows[0])}. Trimming headers.")
    header_names = header_names[:len(rows[0])]

df = pd.DataFrame(rows, columns=header_names)

df.to_csv('nba_stats_2020_21.csv', index=False)

driver.quit()

print(f"Scraped {len(rows)} players. Data saved to 'nba_stats_2020_21.csv'")